# Trabajo final data mining

En este trabajo brindaremos un chat bot de respuestas cortas para IM en español. Primero que nada definiremos una respuesta corta como cualquier turno hasta 3 palabras que ocurren al menos n veces. Iremos probando diferentes n a modo de prueba. El texto anterior a la respuesta corta será llamado contexto

## Extracción del corpus y formación del conjunto de datos

Debido a que no hay un corpus bien definido para este problema utilizaremos el contexto del chat como texto de entrenamiento, por lo que la respuesta corta será la etiqueta. Utilizaremos mensajes de whatsapp tomadas de nuestras conversaciones.

### Procesamiento

In [2]:
import re
import csv
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta
from string import punctuation
from collections import defaultdict

In [3]:
files = glob.glob('todos/*')
raw_data = []
for file in files:
    with open(file, 'r') as f:
        raw_data += [f.readlines()]

Debemos quitar líneas que no son parte de los chats. Todos contienen fecha y hora.

`6/18/17, 16:43 - Los mensajes y llamadas en este chat ahora están protegidos con cifrado de extremo a extremo. Toca para más información.`

`3/22/17, 19:12 - Mati created group "Discreta II"\n`

`4/6/15, 22:44 - You were added\n`

`2/1/17, 01:41 - NN's security code changed. Tap for more info.\n`

In [4]:
raw_data[0]

['10/16/17, 17:44 - Messages to this chat and calls are now secured with end-to-end encryption. Tap for more info.\n',
 '10/16/17, 17:44 - \u202a+54 9 351 328-5957\u202c: https://m.facebook.com/story.php?story_fbid=1490068964433492&id=161872773919791\n',
 '10/16/17, 17:54 - Daniela Bosch: Hola Te confundiste de numero\n']

In [5]:
# Veamos cómo se guardan los mensajes de múltiples líneas.
raw_data[1][1:6]

['29/9/17, 16:59 - Ori creó el grupo "Libertad 🗽 🌏"\n',
 '6/10/17, 16:57 - Ori te añadió\n',
 '6/10/17, 16:57 - Ahora eres un administrador\n',
 '6/10/17, 16:58 - Ori: 😂😂😂😂😂😂😂😂😂😂\n',
 '6/10/17, 17:43 - Luciano: Que es este lugar 👀\n']

A pesar que tiene varias líneas este mensaje, debería ser __parte del mismo__.

In [7]:
raw_data[0][0]

'10/16/17, 17:44 - Messages to this chat and calls are now secured with end-to-end encryption. Tap for more info.\n'

Además esta claro que está __información extra__ (denominada meta) en los mensajes de nada sirve.

Luego para resumir un poco los problemas podemos nombras los siguientes:
- __mensaje de varias líneas__: unir todos como un sólo mensaje. funciones: is_continuation e is_message.
- __meta información en el mensaje__: todo mensaje de bienvenida

Es claro que pertenece al mismo mensaje, sin embargo esta divido por \n. 

La solución planteada fue agregar todo al mismo mensaje (is_continuation).

#### Formato lindo

Para ello spliteo cada línea por la primera ocurrencia de ": ". Ver bien si se puede hacer algo con las líneas que no tengan el mismo formato (por ahora la quitamos). Hay un problema con las fechas debido a la configuración de cada telefono. Algunos formatos es mm/dd/yyyy y otros dd/mm/yyyy. Veremos como solucionarlo en un futuro.

In [9]:
def is_message(text):
    """
    No es un mensaje de varias líneas.
    """
    return re.search(r'\d+/\d+/\d+, \d+:\d+.+ - .+: ', text) is not None
    # return ': ' in text and '/' in text and ',' in text

def is_continuation(text):
    """
    Pertenece al mismo mensaje anterior.
    """
    return re.search(r'\d+/\d+/\d+, \d+:\d+.+ - ', text) is None
    # return '/' not in text or ': ' not in text or ',' not in text

def is_meta(text):
    """
    Contiene metadatos sobre el chat.
    """
    return re.search(r'\d+/\d+/\d+, \d+:\d+.+ - ', text) is not None and ': ' not in text

# at the begining we don't filter any kind of message. Let's see what happend.
info_chat = []
for i, chat in enumerate(raw_data):
    users = {}
    for line in chat:
        line = line.strip('\n')
        if is_meta(line):
            continue
        if is_continuation(line):
            info_chat[len(info_chat)-1][4] += ". " + line
            continue
        if is_message(line):
            info, text = line.split(": ", 1)
            datetime, owner = info.split(" - ", 1)
            date, time = datetime.split(", ", 1)
            
            # anonymizing users
            if owner not in users:
                users[owner] = str(len(users))
            user = "USER" + users[owner]
                
            info_chat.append([i, user, date, time, text])

Ahora guardaremos el corpus en formato csv para hacerlo legible. 

Tendrá cinco columnas: numero de chat, persona que escribe el msj, fecha y hora en que se envío, mensaje.

| idchat ||  Owner  ||     Date    || hour ||    Message     |
|--------||---------||-------------||------||----------------|
|    1   ||  USER0  || 20-05-2017  || 20:10|| hola que haces |
|    1   ||  USER1  || 21-05-2017  || 12:20|| Todo bien?     |
|    2   ||  USER0  || 15-07-2017  || 13:15||Me fui al baile |

In [10]:
info_chat[:10]

[[1, 'USER0', '1/27/16', '12:24', '<Media omitted>'],
 [1,
  'USER0',
  '2/4/16',
  '09:50',
  'Lee Esta Historia!          . Una mujer muy enferma soñó que Jesús le dio a beber agua.. . Cuando se despertó por la mañana estaba bien otra vez, en forma, bien y curada.. . Ella vio un trozo de papel al lado de su mesa que decía: "Jesús es el verdadero Dios viviente".. . Le dijo a la gente acerca de lo que le había sucedido a ella.. . Un oficial que escucho la historia de la dama,  y al instante fue promovido en el trabajo.. . Otro hombre que recibio el texto elimino el mensaje de inmediato y este  sufrio grandes pérdidas durante 13 dias.. . Veras como Jesus hara maravilla yo si lo creo porque Dios existe... ¡Amen!. . "Hola soy Jesucristo, se que casi no tienes tiempo para mi... Te Amo y te Bendigo siempre estoy contigo. Hoy quiero que  este mensaje recorra todo el mundo antes de la media noche por favor no la cortes y te voy a ayudar con algo que tu estas necesitando... Amén.        . Mand

In [11]:
fieldnames = ['id', 'owner', 'date', 'hour', 'message']
with open('corpus.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    
    for idc, owner, date, hour, message in info_chat:
        if message =='':
            continue
        writer.writerow({'id': idc, 'owner': owner, 'date': date, 'hour': hour, 'message': message})